In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import sys
import random as rd
import torch
from torch import nn, optim
import torch.nn.functional as F
import torch.utils.data as tud
# from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

import CustomDataset as cd
import Net

### Loading Data

First, I'm going to try to distinguish between Carbon 130keV and Flourine 170keV.

In [2]:
C130_dat, F170_dat = np.load('data/C_130keV.npy'), np.load('data/F_170keV.npy')

Now I'm going to put them all in one file and shuffle.

In [3]:
C130_dat.shape

(5000, 97, 97)

In [4]:
F170_dat.shape

(5000, 97, 97)

For a learning algorithm, it will be necesarry to have an input (X data) containing a number of $97\times97$ matrices for each energy. The output will be a label, either Carbon or Flourine. The model will compare its output with the actual labels and adjust accordingly to minimise loss. 

In order to quantify each label, the common 'one-hot encoding' method will be used. C elements will have the label `[1, 0]` and F will be `[0, 1]`. Tuples of this form will be the ground truth labels and the model output. I will have to ensure that the model outputs an array of shape `[2]`.

In [5]:
data, labels = [],[]

for i in range(len(C130_dat)):
    data.append(C130_dat[i])
    labels.append([1,0])

for i in range(len(F170_dat)):
    data.append(F170_dat[i])
    labels.append([0,1])

In [6]:
print(np.shape(data))
print(np.shape(labels))

(10000, 97, 97)
(10000, 2)


### Defining Model Training Functions

In [7]:
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

In [8]:
xtrain_tensor = torch.from_numpy(np.array(X_train)) #converts numpy array to torch tensor
xtrain_sqz = xtrain_tensor.unsqueeze(1) #adds new dimension along axis 1
print(xtrain_sqz.shape)


ytrain_tensor = torch.from_numpy(np.array(y_train))
ytrain_sqz = ytrain_tensor.unsqueeze(1)
print(ytrain_sqz.shape)

torch.Size([8000, 1, 97, 97])
torch.Size([8000, 1, 2])


In [9]:
xtest_tensor = torch.from_numpy(np.array(X_test)) #converts numpy array to torch tensor
xtest_sqz = xtest_tensor.unsqueeze(1) #adds new dimension along axis 1
print(xtest_sqz.shape)

ytest_tensor = torch.from_numpy(np.array(y_test))
ytest_sqz = ytest_tensor.unsqueeze(1)
print(ytest_sqz.shape)

torch.Size([2000, 1, 97, 97])
torch.Size([2000, 1, 2])


In [10]:
train_data = cd.CustomDataset(xtrain_sqz.float(), ytrain_sqz.float())
test_data = cd.CustomDataset(xtest_sqz.float(), ytest_sqz.float())

#defining dataloader class
train_loader = tud.DataLoader(dataset=train_data, batch_size=50, shuffle=True)
train_loader_iter = iter(train_loader)

#same as above but for test data
test_loader = tud.DataLoader(dataset=test_data, batch_size=50, shuffle=True)
test_loader_iter = iter(test_loader)

### Train/Test Functions

In [11]:
# def train(model, device, train_loader, optimizer, epoch, batch_size = 50):
#     model.train()
#     #loop run over data output by loader to train model
#     for batch_idx, (data, target) in enumerate(train_loader):
#         data, target = data.to(device), target.to(device)
#         optimizer.zero_grad()
#         output = model(data) #applies model to data
#         # print(target.shape)
#         target = target.squeeze(1) #removes dimension from target
#         # print(target.shape)
#         # print(output)
#         # print(target.shape)
#         loss = nn.CrossEntropyLoss()(output, target[:,1].long()) #calculates cross entropy loss
    
# #         loss = nn.CrossEntropyLoss()(output, target.long())
#         # print(loss)
#         model.zero_grad()
#         loss.backward()
#         optimizer.step() #new step in optimisation of loss
#         #printing output of each batch for loss observation while model is training
#         if batch_idx % batch_size == 0:
#             print(
#                 'Train Epoch: {} [{}/{} ({}%)]\tLoss: {}'.format(
#                     epoch, batch_idx * len(data), len(train_loader.dataset), \
#                     round(100 * batch_idx / len(train_loader),0), round(loss.item(),5)
#                 )
#             )

In [12]:
# def test(model, device, test_loader):
#     model.eval()
#     #initialisting parameters and empty lists
#     test_loss = 0
#     correct = 0
#     probs_pred = []
#     labels_actual = []
#     labels_pred = []
#     with torch.no_grad():
#         for data, target in test_loader:
#             data, target = data.to(device), target.to(device)
#             output = model(data)
#             target=target.squeeze(1)
#             # print(output.shape)
#             test_loss += nn.CrossEntropyLoss()(output, target[:,1].long()).item()
#             # print(output)
# #             test_loss += nn.CrossEntropyLoss()(output, target.long()).item()

#             # pred = output.max(1, keepdim=True)[1] #finds index of maximum in output vector as described below

#             pred = output.max(1, keepdim=False)[1]#torch.round(output)

#             # print(output)
#             # pred_onehot = F.one_hot(pred).squeeze(1) #converts into one-hot predicted label 
#             # print(pred_onehot)
#             # print(test_loss)
#             # pred_bool = torch.eq(pred_onehot,target) #finds values where predicted equals target
#             pred_bool = torch.eq(pred,target)
#             # print(pred_bool)
#             # correct += int((pred_bool.sum().item())/2) #sums number of correct values
#             correct += pred_bool.sum().item()
#             #adds predicted labels, targets and probabilities to lists for later use
#             labels_actual.append(target.cpu().numpy())
#             probs_pred.append(output.cpu().numpy())
#             # labels_pred.append(pred_onehot.cpu().numpy())

#     #printing accuracy and loss after each epoch for analysis
#     test_loss /= len(test_loader.dataset)
#     print(
#         '\nTest set: Average loss: {}, Accuracy: {}/{} ({}%)\n'.format(
#               round(test_loss,4), correct, len(test_loader.dataset), \
#             round(100 * correct / len(test_loader.dataset),0)
#             )
#           )
  
#     return(labels_actual,probs_pred,labels_pred)

In [24]:
batch_size=20

def train(model, device, train_loader, optimizer, epoch):
    model.train()
    #loop run over data output by loader to train model
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data) #applies model to data
        target = target.squeeze(1) #removes dimension from target
        loss = nn.CrossEntropyLoss()(output, target) #calculates cross entropy loss
        loss.backward()
        optimizer.step() #new step in optimisation of loss
        #printing output of each batch for loss observation while model is training
        if batch_idx % batch_size == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100 * batch_idx / len(train_loader), loss.item()))

def test(model, device, test_loader):
    model.eval()
    #initialisting parameters and empty lists
    test_loss = 0
    correct = 0
    probs_pred = []
    labels_actual = []
    labels_pred = []
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            target=target.squeeze(1)
            test_loss += nn.CrossEntropyLoss()(output, target).item()
        
            pred = output.max(1, keepdim=True)[1] #finds index of maximum in output vector as described below
            pred_onehot = F.one_hot(pred).squeeze(1) #converts into one-hot predicted label 
            pred_bool = torch.eq(pred_onehot,target) #finds values where predicted equals target
            correct += int((pred_bool.sum().item())/2) #sums number of correct values
            #adds predicted labels, targets and probabilities to lists for later use
            
            labels_actual.append(target.cpu().numpy())
            probs_pred.append(output.cpu().numpy())
            labels_pred.append(pred_onehot.cpu().numpy())
 
    #printing accuracy and loss after each epoch for analysis
    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
    100 * correct / len(test_loader.dataset)))
    
    return(labels_actual,probs_pred,labels_pred)

In [25]:
device = torch.device('cuda')

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.conv_layers = nn.Sequential(
            nn.Conv2d(1, 5, kernel_size=(5,5)),
            nn.MaxPool2d(kernel_size=(3,3)),
            nn.ReLU(),
            nn.Conv2d(5, 10, kernel_size=(5,5)),
            nn.MaxPool2d(kernel_size=(3,3)),
            nn.ReLU(),
        )
            
        self.fc_layers = nn.Sequential(
            nn.Linear(810, 80),
            nn.ReLU(),
            nn.Linear(80,2),
            # nn.Softmax(dim=1),
            nn.Sigmoid(),
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = x.view(-1, 810) #flattens as described below
#         x = x.view(x.size(0), -1)
        x = self.fc_layers(x)
        return x

In [43]:
model = Net().to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [44]:
#outputting variables from initial model
initial_model = model
print(model)
numel_list = [p.numel() for p in model.parameters()]
sum(numel_list), numel_list

Net(
  (conv_layers): Sequential(
    (0): Conv2d(1, 5, kernel_size=(5, 5), stride=(1, 1))
    (1): MaxPool2d(kernel_size=(3, 3), stride=(3, 3), padding=0, dilation=1, ceil_mode=False)
    (2): ReLU()
    (3): Conv2d(5, 10, kernel_size=(5, 5), stride=(1, 1))
    (4): MaxPool2d(kernel_size=(3, 3), stride=(3, 3), padding=0, dilation=1, ceil_mode=False)
    (5): ReLU()
  )
  (fc_layers): Sequential(
    (0): Linear(in_features=810, out_features=80, bias=True)
    (1): ReLU()
    (2): Linear(in_features=80, out_features=2, bias=True)
    (3): Sigmoid()
  )
)


(66432, [125, 5, 1250, 10, 64800, 80, 160, 2])

In [45]:
num_epochs = 7 #setting number of epochs for CNN

#running loop training and testing over previously specified number of epochs
for epoch in range(1, num_epochs + 1):
    train(model, device, train_loader, optimizer, epoch)
    act,pred,pred_labs = test(model, device, test_loader)

Train Epoch: 1 [0/8000 (0%)]	Loss: 0.745514
Train Epoch: 1 [1000/8000 (12%)]	Loss: 0.732649
Train Epoch: 1 [2000/8000 (25%)]	Loss: 0.664633
Train Epoch: 1 [3000/8000 (38%)]	Loss: 0.637937
Train Epoch: 1 [4000/8000 (50%)]	Loss: 0.703273
Train Epoch: 1 [5000/8000 (62%)]	Loss: 0.696314
Train Epoch: 1 [6000/8000 (75%)]	Loss: 0.671585
Train Epoch: 1 [7000/8000 (88%)]	Loss: 0.680831

Test set: Average loss: 0.0133, Accuracy: 1191/2000 (60%)

Train Epoch: 2 [0/8000 (0%)]	Loss: 0.664653
Train Epoch: 2 [1000/8000 (12%)]	Loss: 0.662531
Train Epoch: 2 [2000/8000 (25%)]	Loss: 0.625596
Train Epoch: 2 [3000/8000 (38%)]	Loss: 0.647038
Train Epoch: 2 [4000/8000 (50%)]	Loss: 0.640330
Train Epoch: 2 [5000/8000 (62%)]	Loss: 0.758945
Train Epoch: 2 [6000/8000 (75%)]	Loss: 0.622727
Train Epoch: 2 [7000/8000 (88%)]	Loss: 0.646239

Test set: Average loss: 0.0130, Accuracy: 1234/2000 (62%)

Train Epoch: 3 [0/8000 (0%)]	Loss: 0.665085
Train Epoch: 3 [1000/8000 (12%)]	Loss: 0.727411
Train Epoch: 3 [2000/8000 (2

### Different Networks

In [ ]:
device = torch.device('gpu')

#defining CNN class, see description below
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.conv_layers = nn.Sequential(
            nn.Conv2d(1, 5, kernel_size=(6,6)),
            nn.MaxPool2d(kernel_size=(2,2)),
            nn.ReLU(),
            nn.Conv2d(5, 10, kernel_size=(5,5)),
            nn.MaxPool2d(kernel_size=(2,2)),
            nn.ReLU(),
            nn.Conv2d(10, 50, kernel_size=(6,6)),
            nn.MaxPool2d(kernel_size=(2,2)),
            nn.ReLU(),
        )
            
        self.fc_layers = nn.Sequential(
            nn.Linear(3200,1000),
            nn.ReLU(),
            nn.Linear(1000,300),
            nn.ReLU(),
            nn.Linear(300,80),
            nn.ReLU(),
            nn.Linear(80,2),
            nn.Softmax(dim=1),
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = x.view(-1, 3200) #flattens as described below
#         x = x.view(x.size(0), -1)
        x = self.fc_layers(x)
        return x

In [33]:
model = Net().to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-4)

In [34]:
#outputting variables from initial model
initial_model = model
numel_list = [p.numel() for p in model.parameters()]

In [36]:
num_epochs = 15 #setting number of epochs for CNN

#running loop training and testing over previously specified number of epochs
for epoch in range(1, num_epochs + 1):
    train(model, device, train_loader, optimizer, epoch)
    act,pred,pred_labs = test(model, device, test_loader)

Train Epoch: 1 [0/8000 (0%)]	Loss: 0.644744
Train Epoch: 1 [1000/8000 (12%)]	Loss: 0.648438
Train Epoch: 1 [2000/8000 (25%)]	Loss: 0.626321
Train Epoch: 1 [3000/8000 (38%)]	Loss: 0.594371
Train Epoch: 1 [4000/8000 (50%)]	Loss: 0.597040
Train Epoch: 1 [5000/8000 (62%)]	Loss: 0.656058
Train Epoch: 1 [6000/8000 (75%)]	Loss: 0.661895
Train Epoch: 1 [7000/8000 (88%)]	Loss: 0.630850

Test set: Average loss: 0.0130, Accuracy: 1235/2000 (62%)

Train Epoch: 2 [0/8000 (0%)]	Loss: 0.616872
Train Epoch: 2 [1000/8000 (12%)]	Loss: 0.617932
Train Epoch: 2 [2000/8000 (25%)]	Loss: 0.573998
Train Epoch: 2 [3000/8000 (38%)]	Loss: 0.640386
Train Epoch: 2 [4000/8000 (50%)]	Loss: 0.665086
Train Epoch: 2 [5000/8000 (62%)]	Loss: 0.650236
Train Epoch: 2 [6000/8000 (75%)]	Loss: 0.629304
Train Epoch: 2 [7000/8000 (88%)]	Loss: 0.633588

Test set: Average loss: 0.0130, Accuracy: 1258/2000 (63%)

Train Epoch: 3 [0/8000 (0%)]	Loss: 0.655050
Train Epoch: 3 [1000/8000 (12%)]	Loss: 0.695628
Train Epoch: 3 [2000/8000 (2

### Now on all the data

In [62]:
im_dat, el_labs, en_labs = [],[],[]
for i in np.arange(50,196,5):
    C_dat = np.load('data/C_'+str(i)+'keV.npy')
    for n in range(np.shape(C_dat)[0]):
        im_dat.append(C_dat[n]),el_labs.append([1,0]),en_labs.append(i)
    F_dat = np.load('data/F_'+str(i)+'keV.npy')
    for n in range(np.shape(F_dat)[0]):
        im_dat.append(F_dat[n]),el_labs.append([0,1]),en_labs.append(i)

In [67]:
X_train, X_test, y_train, y_test = train_test_split(im_dat, el_labs, test_size=0.2, random_state=42)

In [68]:
xtrain_tensor = torch.from_numpy(np.array(X_train)) #converts numpy array to torch tensor
xtrain_sqz = xtrain_tensor.unsqueeze(1) #adds new dimension along axis 1
print(xtrain_sqz.shape)

ytrain_tensor = torch.from_numpy(np.array(y_train))
ytrain_sqz = ytrain_tensor.unsqueeze(1)
print(ytrain_sqz.shape)

xtest_tensor = torch.from_numpy(np.array(X_test)) #converts numpy array to torch tensor
xtest_sqz = xtest_tensor.unsqueeze(1) #adds new dimension along axis 1
print(xtest_sqz.shape)

ytest_tensor = torch.from_numpy(np.array(y_test))
ytest_sqz = ytest_tensor.unsqueeze(1)
print(ytest_sqz.shape)

train_data = cd.CustomDataset(xtrain_sqz.float(), ytrain_sqz.float())
test_data = cd.CustomDataset(xtest_sqz.float(), ytest_sqz.float())

#defining dataloader class
train_loader = tud.DataLoader(dataset=train_data, batch_size=50, shuffle=True)
train_loader_iter = iter(train_loader)

#same as above but for test data
test_loader = tud.DataLoader(dataset=test_data, batch_size=50, shuffle=True)
test_loader_iter = iter(test_loader)

torch.Size([240000, 1, 97, 97])
torch.Size([240000, 1, 2])
torch.Size([60000, 1, 97, 97])
torch.Size([60000, 1, 2])


In [69]:
num_epochs = 5 #setting number of epochs for CNN
batch_size=1000

#running loop training and testing over previously specified number of epochs
for epoch in range(1, num_epochs + 1):
    train(model, device, train_loader, optimizer, epoch)
    act,pred,pred_labs = test(model, device, test_loader)

Train Epoch: 1 [0/240000 (0%)]	Loss: 0.665531
Train Epoch: 1 [50000/240000 (21%)]	Loss: 0.592201
Train Epoch: 1 [100000/240000 (42%)]	Loss: 0.565395
Train Epoch: 1 [150000/240000 (62%)]	Loss: 0.541389
Train Epoch: 1 [200000/240000 (83%)]	Loss: 0.544530

Test set: Average loss: 0.0107, Accuracy: 45951/60000 (77%)

Train Epoch: 2 [0/240000 (0%)]	Loss: 0.538176
Train Epoch: 2 [50000/240000 (21%)]	Loss: 0.516556
Train Epoch: 2 [100000/240000 (42%)]	Loss: 0.565749
Train Epoch: 2 [150000/240000 (62%)]	Loss: 0.498627
Train Epoch: 2 [200000/240000 (83%)]	Loss: 0.497355

Test set: Average loss: 0.0105, Accuracy: 46445/60000 (77%)

Train Epoch: 3 [0/240000 (0%)]	Loss: 0.474754
Train Epoch: 3 [50000/240000 (21%)]	Loss: 0.627133
Train Epoch: 3 [100000/240000 (42%)]	Loss: 0.489579
Train Epoch: 3 [150000/240000 (62%)]	Loss: 0.512409
Train Epoch: 3 [200000/240000 (83%)]	Loss: 0.475893

Test set: Average loss: 0.0107, Accuracy: 45978/60000 (77%)

Train Epoch: 4 [0/240000 (0%)]	Loss: 0.464258
Train Epo

### On a smaller subset of data

In [70]:
def dataprep(X,y):
    X_train, X_test, y_train, y_test = train_test_split(im_dat, el_labs, test_size=0.2, random_state=42)

    xtrain_tensor = torch.from_numpy(np.array(X_train)) #converts numpy array to torch tensor
    xtrain_sqz = xtrain_tensor.unsqueeze(1) #adds new dimension along axis 1
    print(xtrain_sqz.shape)

    ytrain_tensor = torch.from_numpy(np.array(y_train))
    ytrain_sqz = ytrain_tensor.unsqueeze(1)
    print(ytrain_sqz.shape)

    xtest_tensor = torch.from_numpy(np.array(X_test)) #converts numpy array to torch tensor
    xtest_sqz = xtest_tensor.unsqueeze(1) #adds new dimension along axis 1
    print(xtest_sqz.shape)

    ytest_tensor = torch.from_numpy(np.array(y_test))
    ytest_sqz = ytest_tensor.unsqueeze(1)
    print(ytest_sqz.shape)

    train_data = cd.CustomDataset(xtrain_sqz.float(), ytrain_sqz.float())
    test_data = cd.CustomDataset(xtest_sqz.float(), ytest_sqz.float())

    #defining dataloader class
    train_loader = tud.DataLoader(dataset=train_data, batch_size=50, shuffle=True)
    train_loader_iter = iter(train_loader)

    #same as above but for test data
    test_loader = tud.DataLoader(dataset=test_data, batch_size=50, shuffle=True)
    test_loader_iter = iter(test_loader)
    
    return train_loader, test_loader

In [71]:
im_dat, el_labs, en_labs = [],[],[]
for i in np.arange(60,101,5):
    C_dat = np.load('data/C_'+str(i)+'keV.npy')
    for n in range(np.shape(C_dat)[0]):
        im_dat.append(C_dat[n]),el_labs.append([1,0]),en_labs.append(i)
    F_dat = np.load('data/F_'+str(i)+'keV.npy')
    for n in range(np.shape(F_dat)[0]):
        im_dat.append(F_dat[n]),el_labs.append([0,1]),en_labs.append(i)

In [72]:
train_loader, test_loader = dataprep(im_dat,el_labs)

torch.Size([72000, 1, 97, 97])
torch.Size([72000, 1, 2])
torch.Size([18000, 1, 97, 97])
torch.Size([18000, 1, 2])


In [74]:
num_epochs = 5 #setting number of epochs for CNN
batch_size=100

#running loop training and testing over previously specified number of epochs
for epoch in range(1, num_epochs + 1):
    train(model, device, train_loader, optimizer, epoch)
    act,pred,pred_labs = test(model, device, test_loader)

Train Epoch: 1 [0/72000 (0%)]	Loss: 0.420094
Train Epoch: 1 [5000/72000 (7%)]	Loss: 0.494878
Train Epoch: 1 [10000/72000 (14%)]	Loss: 0.474741
Train Epoch: 1 [15000/72000 (21%)]	Loss: 0.454288
Train Epoch: 1 [20000/72000 (28%)]	Loss: 0.445500
Train Epoch: 1 [25000/72000 (35%)]	Loss: 0.440105
Train Epoch: 1 [30000/72000 (42%)]	Loss: 0.466547
Train Epoch: 1 [35000/72000 (49%)]	Loss: 0.400906
Train Epoch: 1 [40000/72000 (56%)]	Loss: 0.454307
Train Epoch: 1 [45000/72000 (62%)]	Loss: 0.412948
Train Epoch: 1 [50000/72000 (69%)]	Loss: 0.483312
Train Epoch: 1 [55000/72000 (76%)]	Loss: 0.451508
Train Epoch: 1 [60000/72000 (83%)]	Loss: 0.476227
Train Epoch: 1 [65000/72000 (90%)]	Loss: 0.507730
Train Epoch: 1 [70000/72000 (97%)]	Loss: 0.426466

Test set: Average loss: 0.0091, Accuracy: 15285/18000 (85%)

Train Epoch: 2 [0/72000 (0%)]	Loss: 0.485347
Train Epoch: 2 [5000/72000 (7%)]	Loss: 0.499022
Train Epoch: 2 [10000/72000 (14%)]	Loss: 0.574212
Train Epoch: 2 [15000/72000 (21%)]	Loss: 0.405212
Tr

### Smaller subset of data

In [78]:
im_dat, el_labs, en_labs = [],[],[]
for i in np.arange(85,101,5):
    C_dat = np.load('data/C_'+str(i)+'keV.npy')
    for n in range(np.shape(C_dat)[0]):
        im_dat.append(C_dat[n]),el_labs.append([1,0]),en_labs.append(i)
    F_dat = np.load('data/F_'+str(i)+'keV.npy')
    for n in range(np.shape(F_dat)[0]):
        im_dat.append(F_dat[n]),el_labs.append([0,1]),en_labs.append(i)

In [79]:
train_loader, test_loader = dataprep(im_dat,el_labs)

torch.Size([32000, 1, 97, 97])
torch.Size([32000, 1, 2])
torch.Size([8000, 1, 97, 97])
torch.Size([8000, 1, 2])


In [80]:
num_epochs = 5 #setting number of epochs for CNN
batch_size=100

#running loop training and testing over previously specified number of epochs
for epoch in range(1, num_epochs + 1):
    train(model, device, train_loader, optimizer, epoch)
    act,pred,pred_labs = test(model, device, test_loader)

Train Epoch: 1 [0/32000 (0%)]	Loss: 0.333261
Train Epoch: 1 [5000/32000 (16%)]	Loss: 0.353198
Train Epoch: 1 [10000/32000 (31%)]	Loss: 0.315129
Train Epoch: 1 [15000/32000 (47%)]	Loss: 0.313320
Train Epoch: 1 [20000/32000 (62%)]	Loss: 0.313278
Train Epoch: 1 [25000/32000 (78%)]	Loss: 0.313598
Train Epoch: 1 [30000/32000 (94%)]	Loss: 0.324402

Test set: Average loss: 0.0065, Accuracy: 7923/8000 (99%)

Train Epoch: 2 [0/32000 (0%)]	Loss: 0.313262
Train Epoch: 2 [5000/32000 (16%)]	Loss: 0.336176
Train Epoch: 2 [10000/32000 (31%)]	Loss: 0.321230
Train Epoch: 2 [15000/32000 (47%)]	Loss: 0.313263
Train Epoch: 2 [20000/32000 (62%)]	Loss: 0.314692
Train Epoch: 2 [25000/32000 (78%)]	Loss: 0.313262
Train Epoch: 2 [30000/32000 (94%)]	Loss: 0.313266

Test set: Average loss: 0.0064, Accuracy: 7956/8000 (99%)

Train Epoch: 3 [0/32000 (0%)]	Loss: 0.313350
Train Epoch: 3 [5000/32000 (16%)]	Loss: 0.320822
Train Epoch: 3 [10000/32000 (31%)]	Loss: 0.313268
Train Epoch: 3 [15000/32000 (47%)]	Loss: 0.31419

This very high accuracy is only because the recoil tracks are much longer for Carbon.